# 5 Modeling Distributions

模型分布分为:

- 经验分布(Empirical Distribution):根据观察到的数据
- 理论分布(Theoretical Distribution):基于数学公式

这章主要介绍几种常见的理论分布, 我们可以通过这些分布公式计算概率, 从而可以构建PMF(Probability Mass Functions)(离散数据), CDF(Cumulative Distribution Function)(连续数据)

- 离散分布(Discrete Distribution)
  - 二项式分布(Binomial Distribution):用于模拟一系列成功或失败的次数, 例如击中目标的次数
  - 泊松分布(Poisson Distribution): 用于模拟在固定时间间隔内发生的事件次数, 例如一场比赛中的进球
- 连续分布(Continuous Distribution)
  - 指数分布(Exponential Distribution): 描述事件发生之间的时间间隔, 例如两次进球之间的时间
  - 正态分布(Normal Distribution): 也称高斯分布, 描述呈钟形曲线的数据, 例如婴儿的出生体重
  - 对数正态分布(Lognormal Distribution): 描述其对数值遵循正态分布的数据, 通常用于模拟像成人体重这样有偏斜的连续变量

In [ ]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

## 5.1 The Binomial Distribution

这里采用飞碟射击运动举例. 击中或击不中一个飞碟恰好是一个典型的二项式.


In [ ]:
import numpy as np

In [ ]:
def flip(n, p):
    """
    模拟二项试验（Binomial trials）中的随机结果，
    即生成一个由“成功”和“失败”组成的随机序列
    """
    choices = [1, 0]
    probs = [p, 1 - p]
    return np.random.choice(choices, size=n, p=probs)


def simulate_round(n, p):
    """
    模拟一轮射击, 返回击中次数
    """
    seq = flip(n, p)
    return seq.sum()

这是一个模拟 25 个靶标的回合示例，其中每个靶标的命中概率为 90%。

In [ ]:
np.random.seed(1)
flip(25, 0.9)

如果我们生成更长的序列并计算结果的 `Pmf` ，就能确认 1 和 0 的比例是正确的，至少大致如此。

In [ ]:
from empiricaldist import Pmf

seq = flip(1000, 0.9)
pmf = Pmf.from_seq(seq)
pmf

在一场大型比赛中，假设 200 名参赛者每人射击 5 轮，所有参赛者命中目标的概率相同，均为 `p=0.9` 。我们可以通过调用 `simulate_round` 1000 次来模拟这样的比赛。

In [ ]:
n = 25
p = 0.9
results_sim = [simulate_round(n, p) for _ in range(1000)]

平均得分接近 `22.5` ，这是 `n` 和 `p` 的乘积。

In [ ]:
np.mean(results_sim), n * p

以下是结果分布的情况。

In [ ]:
from empiricaldist import Pmf
from thinkstats import decorate

pmf_sim = Pmf.from_seq(results_sim, name="simulation results")

pmf_sim.bar()
decorate(xlabel="Hits", ylabel="PMF")

峰值接近均值，且分布向左偏斜。

我们本可以预测这一分布，而无需进行模拟。从数学上讲，这些结果的分布遵循二项分布，其概率质量函数易于计算。

二项式分布（二项分布）的概率质量函数公式为：

$$P(X = k) = C_n^k p^k (1-p)^{n-k}$$

或写作：

$$P(X = k) = \binom{n}{k} p^k q^{n-k}$$

其中：
- $P(X = k)$ 表示在 $n$ 次独立重复试验中，事件恰好发生 $k$ 次的概率
- $n$ 为试验总次数（$n \geq 1$ 的整数）
- $k$ 为成功次数（$k = 0, 1, 2, \ldots, n$）
- $p$ 为单次试验中事件发生的概率（$0 \leq p \leq 1$）
- $q = 1 - p$ 为单次试验中事件不发生的概率
- $C_n^k$ 或 $\binom{n}{k}$ 为组合数，表示从 $n$ 次试验中选出 $k$ 次成功的组合方式数，计算公式为 $\frac{n!}{k!(n-k)!}$

**公式含义**：该公式描述了在 $n$ 次独立伯努利试验中，成功次数为 $k$ 的概率分布。二项分布的期望值 $E(X) = np$，方差 $D(X) = np(1-p)$。

**典型应用场景**：抛硬币正面朝上的次数、产品合格率检验、投篮命中次数等"成功/失败"类型的重复独立试验问题。

---

**与泊松分布的关系**：当 $n$ 很大而 $p$ 很小时（通常 $n \geq 20$，$p \leq 0.05$），二项分布可用泊松分布近似，其中 $\lambda = np$。这就是泊松定理，也是为什么泊松分布常用于描述稀有事件的原因。

In [ ]:
from scipy.special import comb


def binomial_pmf(k, n, p):
    """
    计算二项分布的概率质量函数
    """
    return comb(n, k) * (p**k) * ((1 - p) ** (n - k))

`binomial_pmf` 函数在给定 `p` 的条件下，计算 `n` 次尝试中获得 `k` 次命中的概率。若我们使用一系列 k 值调用此函数，便能生成一个表示结果分布的 `Pmf` 。

In [ ]:
ks = np.arange(16, n + 1)
ps = binomial_pmf(ks, n, p)
pmf_binom = Pmf(ps, ks, name="binomial model")
ks, ps

In [ ]:
from thinkstats import two_bar_plots

two_bar_plots(pmf_sim, pmf_binom)
decorate(xlabel="Hits", ylabel="PMF")

两者结果相似，仅因模拟结果的随机波动存在细微差异。这种一致性并不令人意外，因为模拟和模型基于相同的假设——特别是每次尝试具有相同成功概率的前提。对模型更严格的检验在于其与实际数据的吻合程度。

从 2020 年夏季奥运会男子双向飞碟射击比赛的维基百科页面，我们可以提取一张显示资格赛结果的表格。

In [ ]:
filename = "Shooting_at_the_2020_Summer_Olympics_Mens_skeet"
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/" + filename)

In [ ]:
import pandas as pd

tables = pd.read_html(filename)
table = tables[6]
table.head()

每位参赛者对应表格中的一行，每轮比赛占据一列。我们将选取包含这些结果的列，并使用 NumPy 函数 `flatten` 将它们整合到单个数组中。

In [ ]:
columns = ["1", "2", "3", "4", "5"]
results = table[columns].values.flatten()
results

在 30 名参赛者中，我们获得了 150 轮比赛的结果，每轮射击 25 发，总共 3750 次尝试中命中了 3575 次。

In [ ]:
total_shots = n * len(results)
total_hits = results.sum()
n, total_shots, total_hits

因此总体成功率为 95.3%。

In [ ]:
p = total_hits / total_shots
p

现在让我们计算一个 Pmf ，它代表具有 n=25 的二项分布以及我们刚刚计算的 p 值。

In [ ]:
ps = binomial_pmf(ks, n, p)
pmf_binom = Pmf(ps, ks, name="binomial model")

我们可以将其与实际结果的 Pmf 进行比较。

In [ ]:
pmf_results = Pmf.from_seq(results, name="actual results")

two_bar_plots(pmf_results, pmf_binom)
decorate(xlabel="Hits", ylabel="PMF")

二项模型与数据分布高度吻合——尽管它做出了不切实际的假设，即所有竞争者都具备相同且恒定的能力。

## 5.2 The Poisson Distribution

另一个体现体育赛事结果遵循可预测模式的例子是冰球比赛中的进球数。

我们将从模拟一场 60 分钟的比赛开始，即 3600 秒，假设每场比赛平均总共进球 6 个，且在任何一秒内进球的概率 `p` 都相同。

In [ ]:
n = 3600
m = 6
p = m / 3600
p

现在我们可以使用以下函数来模拟 n 秒并返回进球总数。

In [ ]:
def simulate_goals(n, p):
    return flip(n, p).sum()

如果我们模拟多场比赛，可以确认每场比赛的平均进球数接近 6 个。

现在我们模拟1001场比赛, 每场比赛3600秒, 每秒进球 `6 / 3600`

In [ ]:
goals = [simulate_goals(n, p) for i in range(1001)]
np.mean(goals)

我们可以使用二项分布来模拟这些结果，但当比赛场次较多且单场进球概率较低时，泊松分布也能很好地模拟这些结果。泊松分布由一个通常用希腊字母λ表示的值来定义，该字母读作“lambda”，在代码中用变量`lam`表示（`lambda`不能作为合法变量名，因为它是 Python 关键字）。`lam`代表进球率，在本例中为每场比赛 6 个进球。

泊松分布的概率质量函数（PMF）公式为：

$$P(X = k) = \frac{\lambda^k e^{-\lambda}}{k!}$$

其中：
- $P(X = k)$ 表示随机变量 $X$ 取值为 $k$ 的概率
- $\lambda$ 是单位时间（或单位空间）内事件发生的平均次数（$\lambda > 0$）
- $k$ 是实际发生次数（$k = 0, 1, 2, \ldots$）
- $e$ 是自然常数（约等于 2.71828）
- $k!$ 是 $k$ 的阶乘

**公式含义**：该公式描述了在固定时间或空间内，事件发生 $k$ 次的概率，适用于描述**稀有事件**在大量独立试验中的发生情况。泊松分布的期望和方差都等于 $\lambda$。

**典型应用场景**：单位时间内接到的电话呼叫数、网站访问量、机器故障次数、放射性粒子衰变数等低概率、独立发生的事件计数问题。


In [ ]:
from scipy.special import factorial


def poisson_pmf(k, lam):
    """Compute the Poisson PMF.

    k (int or array-like): The number of occurrences
    lam (float): The rate parameter (λ) of the Poisson distribution

    returns: float or ndarray
    """
    return (lam**k) * np.exp(-lam) / factorial(k)

如果我们用一系列 `k` 值调用 `poisson_pmf` ，就能生成一个代表结果分布的 `Pmf` 。

In [ ]:
lam = 6
ks = np.arange(20)
ps = poisson_pmf(ks, lam)
pmf_poisson = Pmf(ps, ks, name="Poisson model")

并确认该分布的均值接近 6。

In [ ]:
pmf_poisson.normalize()
pmf_poisson.mean()

下图将模拟结果与具有相同均值的泊松分布进行了比较。

In [ ]:
pmf_sim = Pmf.from_seq(goals, name="simulation")

two_bar_plots(pmf_sim, pmf_poisson)
decorate(xlabel="Goals", ylabel="PMF")

除了因随机波动造成的微小差异外，这些分布情况基本相似。这并不令人意外，因为模拟和泊松模型都基于同一假设：在比赛的任何一秒内进球的概率是相同的。因此，更严格的检验是看模型与真实数据的拟合程度如何。



我从 HockeyReference 网站下载了 2023-2024 赛季国家冰球联盟（NHL）常规赛（不包括季后赛）的所有比赛结果。我提取了 60 分钟常规比赛时间内的进球信息，不包括加时赛或决胜射门环节。数据保存在一个 HDF 文件中，每场比赛对应一个键值，其中记录了每个进球发生的时间（以比赛开始后的秒数表示）。本章节的笔记本中提供了下载这些数据的详细说明。

In [ ]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/nhl_2023_2024.hdf")

In [ ]:
filename = "nhl_2023_2024.hdf"

with pd.HDFStore(filename, "r") as store:
    keys = store.keys()

len(keys), keys[0]

常规赛季期间共有 1312 场比赛。每个键包含比赛日期和主队的三字母缩写。我们可以使用 read_hdf 来查找键并获取进球得分的时间列表。

In [ ]:
times = pd.read_hdf(filename, key=keys[0])
times

赛季首场比赛中，共打入六球，首球出现在开赛 424 秒后，末球则在比赛仅剩 87 秒时的第 3513 秒攻入。

In [ ]:
3600 - times[5]

以下循环读取所有比赛的结果，统计每场比赛的进球数，并将结果存储在一个列表中。

In [ ]:
goals = []

for key in keys:
    times = pd.read_hdf(filename, key=key)
    n = len(times)
    goals.append(n)

每场比赛的平均进球数略高于 6 个。

In [ ]:
lam = np.mean(goals)
lam

我们可以使用 poisson_pmf 来创建一个 Pmf ，使其代表一个与数据具有相同均值的泊松分布。

In [ ]:
ps = poisson_pmf(ks, lam)
pmf_poisson = Pmf(ps, ks, name="Poisson model")

与数据的概率质量函数相比，其形态如下所示。

In [ ]:
pmf_goals = Pmf.from_seq(goals, name="goals scored")

two_bar_plots(pmf_goals, pmf_poisson)
decorate(xlabel="Goals", ylabel="PMF")

## 5.3 The Exponential Distribution

在上一节中，我们模拟了一个简单的冰球比赛模型，其中进球在比赛的任何一秒内发生的概率相同。在同一模型下，事实证明，直到第一个进球的时间遵循指数分布。

为了演示，我们再次假设各队平均总共进 6 个球，并计算每一秒进球的概率。

In [ ]:
n = 3600
m = 6
p = m / 3600
p

以下函数模拟了 n 秒，并使用 argmax 来找到第一个进球的时间。

In [ ]:
def simulate_first_goal(n, p):
    return flip(n, p).argmax()

这是因为 flip 的结果是一个由 1 和 0 组成的序列，所以最大值几乎总是 1。如果序列中至少有一个目标， argmax 会返回第一个目标的索引。如果没有目标，它会返回 0，但这种情况很少发生，我们可以忽略它。

In [ ]:
np.random.seed(3)

In [ ]:
first_goal_times = [simulate_first_goal(n, p) for i in range(1001)]
mean = np.mean(first_goal_times)
mean

第一个进球出现的平均时间接近 600 秒，也就是 10 分钟。这很合理——如果我们预期每场 60 分钟的比赛有 6 个进球，那么平均每 10 分钟就会有一个进球。

当 `n` 较大且 `p` 较小时，我们可以从数学上证明，直到第一个进球发生的预期时间遵循指数分布。

由于模拟生成了许多独特的时间值，我们将使用累积分布函数而非概率质量函数来比较分布。指数分布的累积分布函数计算起来十分简便。

指数分布的概率密度函数（PDF）公式为：

$$f(x) = \lambda e^{-\lambda x} \quad (x \geq 0)$$

其中：
- $f(x)$ 表示随机变量 $X$ 在 $x$ 处的概率密度
- $\lambda > 0$ 是**率参数**（rate parameter），表示单位时间内事件发生的平均次数
- $e$ 是自然常数（约等于 2.71828）

**累积分布函数（CDF）** 为：

$$F(x) = P(X \leq x) = 1 - e^{-\lambda x} \quad (x \geq 0)$$

**期望和方差**：
- 期望值：$E(X) = \frac{1}{\lambda}$
- 方差：$Var(X) = \frac{1}{\lambda^2}$

---

### 📌 公式含义与特点

指数分布描述的是**无记忆性**的连续随机事件，常用于表示：
- 两个连续事件之间的**等待时间**（如设备故障间隔时间、顾客到达间隔时间）
- 寿命分布（在可靠性分析中）

**关键特性**：
- **无记忆性**：$P(X > s + t \mid X > s) = P(X > t)$，即已经等待了时间 $s$，再等待时间 $t$ 的概率与从零开始等待 $t$ 的概率相同
- **参数 $\lambda$ 的意义**：$\lambda$ 越大，事件发生越频繁，等待时间越短（期望值 $1/\lambda$ 越小）

---

### 🔄 与泊松分布的关系

指数分布与泊松分布密切相关：
- 如果**单位时间内事件发生次数**服从泊松分布（参数 $\lambda$），那么**事件间隔时间**就服从指数分布（参数 $\lambda$）
- 例如：如果每分钟平均有 $\lambda$ 个顾客到达（泊松分布），那么顾客到达间隔时间（分钟）就服从指数分布

---

### 📊 应用场景举例

假设某设备平均每 100 小时发生一次故障（$\lambda = 0.01$），则：
- 设备在 50 小时内发生故障的概率：$P(X \leq 50) = 1 - e^{-0.01 \times 50} \approx 0.393$
- 设备寿命超过 200 小时的概率：$P(X > 200) = e^{-0.01 \times 200} \approx 0.135$

---

### ⚠️ 注意事项

1. **定义域**：$x \geq 0$，负值概率密度为 0
2. **参数 $\lambda$ 与 $\theta$ 的表示**：有时也用 $\theta = 1/\lambda$（尺度参数）表示，此时 PDF 为 $f(x) = \frac{1}{\theta}e^{-x/\theta}$
3. **与伽马分布的关系**：指数分布是形状参数为 1 的伽马分布的特例

如果需要计算具体数值或了解其他分布公式，可以继续提问。

In [ ]:
def exponential_cdf(x, lam):
    """Compute the exponential CDF.

    x: float or sequence of floats
    lam: rate parameter

    returns: float or NumPy array of cumulative probability
    """
    return 1 - np.exp(-lam * x)

`lam` 的值表示单位时间内事件的平均发生次数——在此例中为每秒进球数。我们可以利用模拟结果的平均值来计算 `lam` 。



In [ ]:
lam = 1 / mean
lam

在《Think Stats》第 5.2 节和第 5.3 节中，$\lambda$（Lambda，代码中为 `lam`）在数学本质上确实都代表**单位时间（或空间）内事件发生的平均速率（Rate）**，但它们在两个章节中的**具体应用场景**和**描述对象**有显著区别。

以下是详细的对比分析：

### 1. 描述的对象不同（数量 vs. 时间）
虽然 $\lambda$ 的定义相似，但它们服务的分布类型和回答的问题不同：
*   **5.2 章节（泊松分布）：** 描述的是**“多少个”**。$\lambda$ 代表在一个固定时间段内，事件发生的**平均次数**。
    *   *例子：* 每场比赛平均进 6 个球（$\lambda = 6$）。
    *   *回答：* 这场比赛刚好进 $k$ 个球的概率是多少？
*   **5.3 章节（指数分布）：** 描述的是**“多久”**。$\lambda$ 依然是速率，但此时分布关注的是**两次事件之间的时间间隔**（或到第一个事件发生的时间）。
    *   *例子：* 进球速率是每秒 0.00167 个（$\lambda \approx 0.00167$）。
    *   *回答：* 到达第一个进球需要等待 $x$ 秒的概率（累积概率）是多少？

### 2. 离散与连续的区别
*   **泊松分布（5.2）是离散的（Discrete）：** 进球数必须是整数（0, 1, 2...），因此它使用**概率质量函数（PMF）**来构建映射表。
*   **指数分布（5.3）是连续的（Continuous）：** 时间（或间距）可以是数轴上的任何值（如 597.79 秒），因此它使用**累积分布函数（CDF）**或概率密度函数（PDF）来描述。

### 3. $\lambda$ 与均值（Mean）的关系
这是最容易混淆的地方，两者在数学推导中互为倒数关系：
*   **在泊松分布中：** $\lambda$ **就是**该分布的均值。如果 $\lambda=6$，那么你预期的平均发生次数就是 6。
*   **在指数分布中：** $\lambda$ 是**均值的倒数**。如果进球的平均等待时间是 600 秒，那么速率 $\lambda = 1/600 \approx 0.00167$。

### 4. 它们内在的逻辑联系
作者在 5.3 节中解释了这两者的统一性：
如果单位时间内发生的进球**次数**符合参数为 $\lambda$ 的**泊松分布**，那么这些进球之间的**时间间隔**就一定符合参数为 $\lambda$ 的**指数分布**。

### 总结对比表

| 特征 | 5.2 泊松分布 (Poisson) | 5.3 指数分布 (Exponential) |
| :--- | :--- | :--- |
| **$\lambda$ 的含义** | 平均发生**次数** (Events per period) | 平均发生**速率** (Rate of occurrence) |
| **描述内容** | 事件发生的**数量** | 事件之间的**时间间隔** |
| **分布类型** | **离散** (Discrete) | **连续** (Continuous) |
| **与均值的关系** | $\text{Mean} = \lambda$ | $\text{Mean} = 1/\lambda$ |
| **常用函数** | `poisson_pmf(k, lam)` | `exponential_cdf(x, lam)` |

**回答你的疑问：**
你认为它们都是“固定时间间隔事件发生的次数”，这其实只完整描述了**泊松分布**。在**指数分布**中，时间不再是“固定间隔”，而是变成了我们要观察的变量本身。

In [ ]:
lam = 1 / mean
lam, mean

如果我们用一系列时间值调用这个函数，就能近似得出首次进球时间的分布情况。NumPy 函数 linspace 会创建一个等间距数值数组；在此示例中，它计算了从 0 到 3600（包含两端）的 201 个数值。

In [ ]:
from empiricaldist import Cdf

ts = np.linspace(0, 3600, 201)
ps = exponential_cdf(ts, lam)
cdf_expo = Cdf(ps, ts, name="exponential model")

下图将模拟结果与我们刚刚计算出的指数分布进行了比较。

In [ ]:
cdf_sim = Cdf.from_seq(first_goal_times, name="simulation")

cdf_expo.plot(ls=":", color="gray")
cdf_sim.plot()

decorate(xlabel="Time of first goal (seconds)", ylabel="CDF")

指数模型与模拟结果高度吻合——但更严格的检验是看它在真实数据上的表现。

以下循环读取所有比赛的结果，获取第一个进球的时间，并将结果存储在一个列表中。如果没有进球，它会向列表中添加 nan 。

In [ ]:
filename = "nhl_2023_2024.hdf"

with pd.HDFStore(filename, "r") as store:
    keys = store.keys()

In [ ]:
firsts = []

for key in keys:
    times = pd.read_hdf(filename, key=key)
    if len(times) > 0:
        firsts.append(times[0])
    else:
        firsts.append(np.nan)

为了估算进球率，我们可以使用 `nanmean` ，它计算时间的平均值，忽略 `nan` 值。

In [ ]:
lam = 1 / np.nanmean(firsts)
lam

现在我们可以计算具有与数据相同进球率的指数分布的累积分布函数。

In [ ]:
ps = exponential_cdf(ts, lam)
cdf_expo = Cdf(ps, ts, name="exponential model")

为了计算数据的累积分布函数，我们将使用 `dropna=False` 参数，该参数在末尾包含了 `nan` 值。

In [ ]:
cdf_firsts = Cdf.from_seq(firsts, name="data", dropna=False)
cdf_firsts.tail()

下图将指数分布与数据分布进行了比较。

In [ ]:
cdf_expo.plot(ls=":", color="gray")
cdf_firsts.plot()

decorate(xlabel="Time of first goal (seconds)", ylabel="CDF")

数据在某些地方与模型存在偏差——看起来前 1000 秒内的进球数比模型预测的要少。但总体而言，模型与数据的拟合度仍然良好。

这些模型——进球数的泊松模型与时间的指数模型——的基本假设是，在比赛的任何一秒内进球的概率是相等的。如果你问一个冰球迷这是否属实，他们会说不是，而且他们是对的——现实世界在许多方面都违背了这样的假设。尽管如此，理论分布常常能非常出色地拟合实际数据。

## 5.4 The Normal Distribution

我们在现实世界中测量的许多事物都遵循正态分布，也称为高斯分布或“钟形曲线”。为了探究这些分布的来源，让我们思考一个巨型南瓜的生长模型。假设南瓜每天的生长情况取决于天气：天气恶劣时增重 1 磅，天气一般时增重 2 磅，天气良好时增重 3 磅。同时假设每天出现恶劣、一般或良好天气的概率均等。

我们可以使用以下函数来模拟这个模型 n 天，并返回体重增加的总量。

In [ ]:
def simulate_growth(n):
    choices = [1, 2, 3]
    gains = np.random.choice(choices, n)
    return gains.sum()

> _numpy.random.choice方法在不指定概率参数 p时，默认采用均匀分布（uniform）进行随机抽样；但如果显式设置了参数 p，则会按照自定义的概率分布进行抽样_

现在假设有 1001 个人在不同的地方、不同的天气条件下种植巨型南瓜。如果我们模拟 100 天的生长过程，就会得到 1001 个重量的列表。

In [ ]:
sim_weights = [simulate_growth(100) for i in range(1001)]
m, s = np.mean(sim_weights), np.std(sim_weights)
m, s

> *在上面的南瓜的例子中*
> - *随机变量(Random variable, X): 南瓜每天的增重是一个随机变量, 其值可能是1,2,3*
> - *样本容量(Sample size, n): 模拟 100 天的生长过程*
> - *实验次数/样本数量(Number of sample): 1001 个人在不同的地方,不同的天气条件下种植南瓜*

均值接近 200 磅，标准差约为 8 磅。为了检验体重是否服从正态分布，我们将使用以下函数：该函数接收一个样本，并生成一个 Cdf ，该 Cdf 代表一个与样本具有相同均值和标准差的正态分布，其评估范围从均值下方 4 个标准差到均值上方 4 个标准差。

In [ ]:
from scipy.stats import norm


def make_normal_model(data):
    m, s = np.mean(data), np.std(data)
    low, high = m - 4 * s, m + 4 * s
    qs = np.linspace(low, high, 201)
    ps = norm.cdf(qs, m, s)
    return Cdf(ps, qs, name="normal model")

> *在正态分布的上下文中，选择 m±4s 作为 low 和 high 是一种基于覆盖范围和可视化效率的经验性选择（Heuristic）：*
> - *捕捉绝大部分概率质量： 在正态分布中，绝大部分数据都集中在均值附近。根据统计学原理（及来源中对正态分布特性的应用）：*
>   - *约 68% 的数据位于 ±1 倍标准差内。*
>   - *约 95% 的数据位于 ±2 倍标准差内。*
>   - *约 99.7% 的数据位于 ±3 倍标准差内。*
>   - *当范围扩大到 ±4 倍标准差时，所涵盖的面积（即概率）已经极其接近 1 (100%)。*
> - *可视化“长尾”： 作者在 5.4 节提到，由于现实世界的数据（如婴儿出生体重）可能存在偏离模型的极端值（Outliers），使用 4 倍标准差可以确保图表能够完整展示理论模型的“尾部”，从而更清晰地观察实际数据与模型在极端情况下的差异。*

In [ ]:
cdf_model = make_normal_model(sim_weights)

现在我们可以创建一个 Cdf 来表示模拟结果的分布。

In [ ]:
cdf_sim_weights = Cdf.from_seq(sim_weights, name="simulation")

> *作者同时使用这两种方法是为了进行对比分析。简单来说，一种代表“现实（或模拟）的数据”，另一种代表“理想的数学模型”。*
> 1. *Cdf.from_seq()：构建“经验分布”（Empirical Distribution）. 它是在描述“数据实际上是什么样的”。它不需要假设数据符合任何特定的规律，只是纯粹地反映样本中各数值出现的频率和比例*
> 2. *scipy.stats.norm + Cdf()：构建“理论分布”（Theoretical Distribution）. 它是在描述“如果数据完美符合某种数学规律（如正态分布），它应该是什么样的”*

我们将使用以下函数来比较分布。 cdf_model 和 cdf_data 是 Cdf 对象。 xlabel 是一个字符串，而 options 是一个控制 cdf_data 绘制方式的选项字典。

In [ ]:
def two_cdf_plots(cdf_model, cdf_data, xlabel="", **options):
    cdf_model.plot(ls=":", color="gray")
    cdf_data.plot(**options)
    decorate(xlabel=xlabel, ylabel="CDF")

In [ ]:
two_cdf_plots(cdf_model, cdf_sim_weights, xlabel="Weight (pounds)")

正态模型非常贴合体重的分布规律。总体而言，当我们累加足够多的随机因素时，其总和往往遵循正态分布。这是中心极限定理的必然结果，我们将在第 14 章中再次探讨这一原理。

> ___中心极限定理是概率论中的一项核心原理，它揭示了一个令人惊叹的现象：无论单个事物的原始分布如何，只要从它那里抽取足够多的样本，这些样本的平均值总会形成一个近似正态的分布（即钟形曲线）。这为我们在不确定中把握规律提供了坚实的理论基础.___

> ___中心极限定理既适用于随机变量的和，也适用于随机变量的均值，这两种表述在数学本质上是等价的，只是表现形式不同。___

但首先让我们看看正态分布对真实数据的拟合程度如何。以全国家庭成长调查（NSFG）中的出生体重分布为例，我们可以使用 read_fem_preg 读取数据，然后选择 totalwgt_lb 列，该列记录了以磅为单位的出生体重。

In [ ]:
import nsfg

preg = nsfg.read_fem_preg()
birth_weights = preg["totalwgt_lb"].dropna()

出生体重的平均值约为 7.27 磅，标准差为 1.4 磅，但正如我们所见，这个数据集中存在一些可能是错误的异常值。

In [ ]:
m, s = np.mean(birth_weights), np.std(birth_weights)
m, s

为了减少异常值对估计均值和标准差的影响，我们将使用 SciPy 函数 trimboth 来移除最高值和最低值。

In [ ]:
from scipy.stats import trimboth

trimmed = trimboth(birth_weights.sort_values(), 0.01)
m, s = np.mean(trimmed), np.std(trimmed)
m, s

> *scipy.stats.trimboth 是按按元素数值大小排列后再进行头尾裁剪的.*

经过修剪的数据，均值略有下降，标准差则显著降低。我们将使用修剪后的数据来构建正态模型。

In [ ]:
cdf_model = make_normal_model(trimmed)

并将其与数据的 Cdf 进行比较。

In [ ]:
cdf_birth_weight = Cdf.from_seq(birth_weights, name="data")

two_cdf_plots(cdf_model, cdf_birth_weight, xlabel="Birth weight (pounds)")

正态模型能很好地拟合数据，除了低于 5 磅的部分，那里数据的分布位于模型的左侧——也就是说，最轻的婴儿比我们在正态分布中预期的要轻。现实世界通常比简单的数学模型更为复杂。

## 5.5 The Lognormal Distribution

在前一节中，我们模拟了南瓜的生长，假设南瓜每天增重 1-3 磅，具体取决于天气情况。现在，让我们假设它们的生长与当前重量成正比，即大南瓜每天增重比小南瓜更多——这可能更符合实际情况。

以下函数模拟了这种按比例增长的模式：若天气恶劣，南瓜增重 3%；若天气适中，增重 5%；若天气良好，增重 7%。我们再次假设在任何给定日期，天气为恶劣、适中或良好的概率均等。

In [ ]:
def simulate_proportionate_growth(n):
    choices = [1.03, 1.05, 1.07]
    gains = np.random.choice(choices, n)
    return gains.prod()

若南瓜增重 3%，最终重量即为初始重量与系数 1.03 的乘积。因此，我们可以通过随机选择增长系数并将它们相乘，来计算 100 天后的重量。

我们将调用此函数 1001 次，以模拟 1001 个南瓜的生长过程，并记录它们的最终重量。

In [ ]:
sim_weights = [simulate_proportionate_growth(100) for i in range(1001)]
np.mean(sim_weights), np.std(sim_weights)

平均重量约为 131 磅；标准差约为 21 磅。因此，该模型中的南瓜比前一个模型中的南瓜更小，但变异性更大。

我们可以从数学上证明它们遵循对数正态分布，这意味着重量的对数遵循正态分布。为了验证这一点，我们将计算重量的对数及其均值和标准差。我们可以使用任何底数的对数，但我会使用以 10 为底的对数，因为这使结果更容易解释。

In [ ]:
log_sim_weights = np.log10(sim_weights)
m, s = np.mean(log_sim_weights), np.std(log_sim_weights)
m, s

现在，让我们将对数的分布与具有相同均值和标准差的正态分布进行比较。

In [ ]:
cdf_model = make_normal_model(log_sim_weights)
cdf_log_sim_weights = Cdf.from_seq(log_sim_weights, name="simulation")

two_cdf_plots(cdf_model, cdf_log_sim_weights, xlabel="Pumpkin weight (log10 pounds)")

该模型与模拟结果非常吻合，这正是我们所预期的。

如果人们像南瓜一样，每年的体重变化与当前体重成比例，我们可能会预期成年人体重的分布遵循对数正态分布。让我们来验证一下。

国家慢性病预防与健康促进中心作为行为风险因素监测系统（BRFSS）的一部分，每年进行一次调查。2008 年，他们采访了 414,509 名受访者，询问了他们的人口统计信息、健康状况和健康风险。在他们收集的数据中，包括 398,484 名受访者的体重。下载数据的说明在本章的笔记本中。

In [ ]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/CDBRFS08.ASC.gz")

thinkstats 模块提供了一个函数，用于读取 BRFSS 数据并返回一个 Pandas DataFrame 。

In [ ]:
from thinkstats import read_brfss

brfss = read_brfss()

成年人体重（以千克为单位）记录在 wtkg2 列中。

In [ ]:
adult_weights = brfss["wtkg2"].dropna()
m, s = np.mean(adult_weights), np.std(adult_weights)
m, s

平均值约为 79 公斤。在计算对数之前，我们先看看体重是否遵循正态分布。

In [ ]:
cdf_model = make_normal_model(adult_weights)
cdf_adult_weights = Cdf.from_seq(adult_weights, name="adult weight")

two_cdf_plots(cdf_model, cdf_adult_weights, xlabel="Adult weight (kilograms)")

对于某些目的而言，正态分布或许足以作为这些数据的模型——但让我们看看能否做得更好。

这是对数变换后体重的分布图，以及一个具有相同均值和标准差的正态模型。

In [ ]:
log_adult_weights = np.log10(adult_weights)
cdf_model = make_normal_model(log_adult_weights)

cdf_log_adult_weights = Cdf.from_seq(log_adult_weights, name="log adult weight")

In [ ]:
two_cdf_plots(cdf_model, cdf_log_adult_weights, xlabel="Adult weight (log10 kg)")

正态模型对对数数据的拟合效果优于对原始体重的拟合，这表明比例增长模型比加法增长模型更能准确描述体重增加的过程。

## 5.8 Exercises

### 5.8.1 Exercise 5.1

在 NSFG 受访者文件中， numfmhh 列记录了每位受访者家庭中的“家庭成员数量”。我们可以使用 read_fem_resp 来读取文件，并使用 query 来筛选出受访时年龄在 25 岁或以上的受访者。

In [ ]:
from nsfg import read_fem_resp

resp = read_fem_resp()

In [ ]:
older = resp.query("age >= 25")
num_family = older["numfmhh"].fillna(0)

计算这些年长受访者的 numfmhh 的 Pmf ，并将其与具有相同均值的泊松分布进行比较。泊松模型对数据的拟合程度如何？

题目提到用泊松分布, 那么首先需要模拟符合泊松分布的数据.

确定 `lambda` 的值, 他应该是平均每个人家庭人员的数量

In [ ]:
lam = np.mean(num_family)
lam

利用最大值和最小值构建 `qs`

In [ ]:
qs = np.arange(num_family.min(), num_family.max() + 1)
ps = poisson_pmf(qs, lam)
pmf_poisson = Pmf(ps, qs, name="Poisson model")

根据数据构建经验分布

In [ ]:
pmf_empirical = Pmf.from_seq(num_family, name="empirical")

two_bar_plots(pmf_empirical, pmf_poisson, xlabel="Family size")
decorate(xlabel="Family size", ylabel="PMF")

### 5.8.2 Exercise 5.2

在本章早些时候，我们观察到冰球比赛中首个进球出现的时间遵循指数分布。如果我们关于进球得分的模型是正确的，那么在任何时刻进球的概率都是相等的，无论距离上一个进球已经过去了多长时间。如果这一假设成立，我们预期进球之间的时间间隔同样遵循指数分布。

以下循环再次读取曲棍球数据，计算连续进球之间的时间间隔（如果一场比赛中进球超过一次），并将这些进球间隔时间收集到一个列表中。

In [ ]:
filename = "nhl_2023_2024.hdf"

with pd.HDFStore(filename, "r") as store:
    keys = store.keys()

In [ ]:
intervals = []

for key in keys:
    times = pd.read_hdf(filename, key=key)
    if len(times) > 1:
        intervals.extend(times.diff().dropna())

使用 exponential_cdf 计算具有与观测区间相同均值的指数分布的累积分布函数，并将此模型与数据的累积分布函数进行比较。

In [ ]:
# 1. 计算 λ：使用实际数据的均值
lam = 1 / np.mean(intervals)

# 2. 构建 qs：基于实际数据的范围
qs = np.linspace(np.min(intervals), np.max(intervals), 201)

# 3. 计算指数分布的 CDF
ps = exponential_cdf(qs, lam)
cdf_expo = Cdf(ps, qs, name="exponential model")

# 4. 构建经验分布的 CDF
cdf_intervals = Cdf.from_seq(intervals, name="data")

# 5. 可视化比较
cdf_expo.plot(ls=":", color="gray")
cdf_intervals.plot()
decorate(xlabel="Time interval between goals (seconds)", ylabel="CDF")

### 5.8.3 Exercise 5.3

人类身高的分布更接近正态分布还是对数正态分布？为了找出答案，我们可以从 BRFSS 中选取身高数据，方法如下：

In [ ]:
adult_heights = brfss["htm3"].dropna()
m, s = np.mean(adult_heights), np.std(adult_heights)
m, s

计算这些值的累积分布函数，并与具有相同均值和标准差的正态分布进行比较。接着计算身高的对数，并将对数的分布与正态分布进行对比。基于视觉比较，哪种模型更符合数据？

In [ ]:
# 创建正态分布的累积分布函数
cdf_normal_model = make_normal_model(adult_heights)
cdf_adult_heights = Cdf.from_seq(adult_heights, name="adult height")

# 可视化比较
two_cdf_plots(
    cdf_normal_model,
    cdf_adult_heights,
    xlabel="Adult height (cm)",
    title="Normal Distribution",
)


In [ ]:
# 创建对数正态分布的累积分布函数
log_adult_heights = np.log10(adult_heights)
cdf_lognormal_model = make_normal_model(log_adult_heights)
cdf_log_adult_heights = Cdf.from_seq(log_adult_heights, name="adult height")

# 可视化比较
two_cdf_plots(
    cdf_lognormal_model,
    cdf_log_adult_heights,
    xlabel="Adult height (cm)",
    title="Log-Normal Distribution",
)
